In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
df=pd.read_csv('../input/data-preprocess/dp.csv' , skiprows=[0,1])

In [ ]:
df

In [ ]:
df=df.drop(['CountryCode','1',' CountryName'], axis=1)

In [ ]:
df.columns

In [ ]:
df

In [ ]:
df.shape

# dealing with missing values

In [ ]:
df.isnull().sum()

In [ ]:
df_bfill=df.fillna(method='bfill')

In [ ]:
df_ffill=df.fillna(method='ffill')

In [ ]:
df2=df.copy()

In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer()
df.iloc[:,0:3]=imputer.fit_transform(df.iloc[:,0:3])

In [ ]:
from sklearn.impute import KNNImputer
knn_imputer=KNNImputer(n_neighbors=2)
df.iloc[:,0:3]=knn_imputer.fit_transform(df.iloc[:,0:3])

In [ ]:
df.isnull().sum()

#  dealing with categorical variable

In [ ]:
df=pd.get_dummies(df , columns=['International Visitors'] , drop_first=True)

In [ ]:
df

In [ ]:
df.describe()

# feature scaling

In [ ]:
df_minmax=df.copy()
df_standard=df.copy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mmscaler=MinMaxScaler()
df_minmax.iloc[:,0:3]=pd.DataFrame(mmscaler.fit_transform(df.iloc[: , 0:3]), columns=df.iloc[:,0:3].columns)
df_minmax.head(2)

In [ ]:
from sklearn.preprocessing import StandardScaler
sscaler=StandardScaler()
df_standard.iloc[:,0:3]=pd.DataFrame(sscaler.fit_transform(df.iloc[:,0:3]), columns=df.iloc[:,0:3].columns)
df_standard.head(2)

# dealing with outliers

In [ ]:
fig , axes = plt.subplots(nrows=4 , ncols=1 )
sns.boxplot(df['Population growth'],ax=axes[0])
sns.boxplot(df['Total population'],ax=axes[1])
sns.boxplot(df['Area (sq. km)'] ,ax=axes[2])
sns.boxplot(df['Coronavirus Cases'] ,ax=axes[3])
plt.show()

In [ ]:
from scipy import stats
z_score=np.abs(stats.zscore(df))

In [ ]:
np.where(z_score>3) 
# so the row=0 and column=2
# and row=14 and column=1 
# are outliers

In [ ]:
df_noutlier_z=df[(z_score<3).all(axis=1)]

In [ ]:
df_noutlier_z.shape

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
cut_off = IQR * 1.5
lower, upper = Q1 - cut_off, Q3 + cut_off

In [ ]:
print(df < (Q1 - 1.5 * IQR))  or (df > (Q3 + 1.5 * IQR))

In [ ]:
df_noutlier_iqr=df[((df >= lower) & (df <= upper)).all(axis=1)]
df_noutlier_iqr.shape

# building linear model 

In [ ]:
x=df_standard.drop(['Coronavirus Cases'] , axis=1)
y=df_standard['Coronavirus Cases']

In [ ]:
#linear regression without feature scaling and with outliers - knn imputed and encoded
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(x,y)

In [ ]:
from sklearn import metrics
y_pred=model.predict(x)
print(model.coef_)
print(model.intercept_)
plt.scatter(y_pred,y)
mse=metrics.mean_squared_error(y,y_pred)
rmse=np.sqrt(mse)
print('MSE is :' , mse)
print('RMSE is :', rmse)

In [ ]:
ys=pd.DataFrame({'real':y ,
                 'prediction':y_pred})

# statsmodel

In [ ]:
import statsmodels.api as smf
model_s=smf.OLS(y,x).fit()

In [ ]:
model_s.params

In [ ]:
model_s.pvalues

In [ ]:
model_s.summary()

# polynomial regression

In [ ]:
x=df['Total population']
y=df['Coronavirus Cases']

In [ ]:
plt.scatter(x,y)

In [ ]:
x = x[:, np.newaxis]
y = y[:, np.newaxis]

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
polynomial_features= PolynomialFeatures(degree=2)
x_poly = polynomial_features.fit_transform(x.reshape(-1,1))

In [ ]:
x_poly

In [ ]:
model_p = LinearRegression()
model_p.fit(x_poly, y)
y_poly_pred = model_p.predict(x_poly)

rmse = np.sqrt(metrics.mean_squared_error(y,y_poly_pred))
r2 = metrics.r2_score(y,y_poly_pred)
print(rmse)
print(r2)

In [ ]:
model_p.coef_